In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim

In [43]:
class classifier_LSTM(nn.Module):

    def __init__(self,
                 relup,
                 input_size,
                 lstm_layers,
                 lstm_size,
                 output1_size,
                 output2_size,
                 GPUindex):
        super(classifier_LSTM, self).__init__()
        self.relup = relup
        self.lstm_layers = lstm_layers
        self.lstm_size = lstm_size
        self.GPUindex = GPUindex
        # batch_first = True => input_shape: (batch, seq, feature)
        # Input: input, (h_0, c_0) 
        # => h_0, c_0: (D*num_layers, N, H_0/C_0) inital hidden state, cell state
        # Output: output, (h_n, c_n)
        self.lstm = nn.LSTM(
            input_size, lstm_size, num_layers = 1, batch_first = True)
        self.output1 = nn.Linear(lstm_size, output1_size)
        self.relu = nn.ReLU()
        if output2_size is None:
            self.output2 = None
        else:
            self.output2 = nn.Linear(lstm_size, output2_size)

    def forward(self, x):
        x = torch.tensor(x)
        print(x.size())
        batch_size = x.size(0)
        # lstm_init: (h_0, c_0)
        lstm_init = (torch.zeros(self.lstm_layers, batch_size, self.lstm_size),
                     torch.zeros(self.lstm_layers, batch_size, self.lstm_size))
        if x.is_cuda:
            lstm_init = (lstm_init[0].cuda(self.GPUindex),
                         lstm_init[0].cuda(self.GPUindex))
        lstm_init = (Variable(lstm_init[0]), Variable(lstm_init[1]))
        x = self.lstm(x, lstm_init)[0]
        print(x.shape)
        x = x[-1, :]
        x = self.output1(x)
        if self.relup:
             x = self.relu(x)
        if self.output2 is not None:
            x = self.output2(x)
        return x

In [15]:
model_test =  nn.LSTM(96, 128, num_layers = 1, batch_first = True)

In [33]:
x = torch.zeros(512, 96)
print(x.size())
x = torch.unsqueeze(x, dim=0)
print(x.size())

torch.Size([512, 96])
torch.Size([1, 512, 96])


In [35]:
lstm_init = (torch.zeros(1, 1, 128),
                     torch.zeros(1, 1, 128))

In [36]:
x = model_test(x, lstm_init)

In [44]:
model=classifier_LSTM(relup=True,input_size = 96,
                    lstm_layers = 1,
                    lstm_size = 128,
                    output1_size = 128,
                    output2_size = None,
                    GPUindex = 1)


In [45]:
from pytorch_model_summary import summary

In [46]:
input_tensor = torch.zeros(1, 512, 96)
summary(model,input_tensor)

torch.Size([1, 512, 96])
torch.Size([1, 512, 128])


C:\Users\AI-INNOVATOR\AppData\Local\Temp\ipykernel_12376\3619269690.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)


'-----------------------------------------------------------------------------------------------\n      Layer (type)                                Output Shape         Param #     Tr. Param #\n===============================================================================================\n            LSTM-1     [1, 512, 128], [1, 1, 128], [1, 1, 128]         115,712         115,712\n          Linear-2                                  [512, 128]          16,512          16,512\n            ReLU-3                                  [512, 128]               0               0\n===============================================================================================\nTotal params: 132,224\nTrainable params: 132,224\nNon-trainable params: 0\n-----------------------------------------------------------------------------------------------'